In [1]:
import findspark as fs
fs.init()
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, translate, lower
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql import functions as F
spark=SparkSession.builder.appName('GoogleData').master('local').getOrCreate()
sc=spark.sparkContext
sql=SQLContext(sc)

In [2]:
spark

In [3]:
orders = spark.read.csv('retail_db//orders.csv',header=True,inferSchema=True)
order_items = spark.read.csv('retail_db//order_items.csv',header=True,inferSchema=True)
customers = spark.read.csv('retail_db//customers.csv',header=True,inferSchema=True)
products = spark.read.csv('retail_db//products.csv',header=True,inferSchema=True)

# select

In [4]:
orders.select(orders.order_status,'order_status',col('order_status')).show()

+---------------+---------------+---------------+
|   order_status|   order_status|   order_status|
+---------------+---------------+---------------+
|         CLOSED|         CLOSED|         CLOSED|
|PENDING_PAYMENT|PENDING_PAYMENT|PENDING_PAYMENT|
|       COMPLETE|       COMPLETE|       COMPLETE|
|         CLOSED|         CLOSED|         CLOSED|
|       COMPLETE|       COMPLETE|       COMPLETE|
|       COMPLETE|       COMPLETE|       COMPLETE|
|       COMPLETE|       COMPLETE|       COMPLETE|
|     PROCESSING|     PROCESSING|     PROCESSING|
|PENDING_PAYMENT|PENDING_PAYMENT|PENDING_PAYMENT|
|PENDING_PAYMENT|PENDING_PAYMENT|PENDING_PAYMENT|
| PAYMENT_REVIEW| PAYMENT_REVIEW| PAYMENT_REVIEW|
|         CLOSED|         CLOSED|         CLOSED|
|PENDING_PAYMENT|PENDING_PAYMENT|PENDING_PAYMENT|
|     PROCESSING|     PROCESSING|     PROCESSING|
|       COMPLETE|       COMPLETE|       COMPLETE|
|PENDING_PAYMENT|PENDING_PAYMENT|PENDING_PAYMENT|
|       COMPLETE|       COMPLETE|       COMPLETE|


#### But while using functions it is applicable to specify the full notation, i.e., dataframe.column_name or using col as below

In [12]:
orders.select(lower(orders.order_status),lower(col('order_status')),lower('order_status')).show()

+-------------------+-------------------+-------------------+
|lower(order_status)|lower(order_status)|lower(order_status)|
+-------------------+-------------------+-------------------+
|             closed|             closed|             closed|
|    pending_payment|    pending_payment|    pending_payment|
|           complete|           complete|           complete|
|             closed|             closed|             closed|
|           complete|           complete|           complete|
|           complete|           complete|           complete|
|           complete|           complete|           complete|
|         processing|         processing|         processing|
|    pending_payment|    pending_payment|    pending_payment|
|    pending_payment|    pending_payment|    pending_payment|
|     payment_review|     payment_review|     payment_review|
|             closed|             closed|             closed|
|    pending_payment|    pending_payment|    pending_payment|
|       

# alias
#### alias should be enclosed within select 

### Note : Please do not give alias the same name as built-in functions 

In [20]:
#orders.select(orders.order_status.alias('Status_alias')).show()
#Note: orders.select('order_status').alias('Status_alias').show()  will not work
orders.select(lower(orders.order_status).alias('1'),lower(col('order_status')).alias('2'),lower('order_status').alias('3'))

DataFrame[1: string, 2: string, 3: string]

## Distinct

In [19]:
orders.select(lower(orders.order_status).alias('1'),lower(col('order_status')).alias('2'),lower('order_status').alias('3')).distinct().show()

+---------------+---------------+---------------+
|              1|              2|              3|
+---------------+---------------+---------------+
|        on_hold|        on_hold|        on_hold|
|     processing|     processing|     processing|
|       canceled|       canceled|       canceled|
|        pending|        pending|        pending|
|         closed|         closed|         closed|
| payment_review| payment_review| payment_review|
|       complete|       complete|       complete|
|suspected_fraud|suspected_fraud|suspected_fraud|
|pending_payment|pending_payment|pending_payment|
+---------------+---------------+---------------+



# withcolumn
#### If already existing column name is given, then the new column expression will replace the existing column. (It will not add another column)


In [25]:
orders = orders. \
withColumn('order_id',orders.order_id.cast('bigint')). \
withColumn('order_date',orders.order_date.cast('date')). \
withColumn('order_customer_id',orders.order_customer_id.cast('bigint')). \
withColumn('order_status',col('order_status').cast('string')) 


In [7]:
orders.select('order_id','order_status').withColumn('SomethingNew',orders.order_id+1).show()

+--------+---------------+------------+
|order_id|   order_status|SomethingNew|
+--------+---------------+------------+
|       1|         CLOSED|           2|
|       2|PENDING_PAYMENT|           3|
|       3|       COMPLETE|           4|
|       4|         CLOSED|           5|
|       5|       COMPLETE|           6|
|       6|       COMPLETE|           7|
|       7|       COMPLETE|           8|
|       8|     PROCESSING|           9|
|       9|PENDING_PAYMENT|          10|
|      10|PENDING_PAYMENT|          11|
|      11| PAYMENT_REVIEW|          12|
|      12|         CLOSED|          13|
|      13|PENDING_PAYMENT|          14|
|      14|     PROCESSING|          15|
|      15|       COMPLETE|          16|
|      16|PENDING_PAYMENT|          17|
|      17|       COMPLETE|          18|
|      18|         CLOSED|          19|
|      19|PENDING_PAYMENT|          20|
|      20|     PROCESSING|          21|
+--------+---------------+------------+
only showing top 20 rows



#### Given below are the cast notations from pyspark.sql.types

In [ ]:
BinaryType: binary
BooleanType: boolean
ByteType: tinyint
DateType: date
DecimalType: decimal(10,0)
DoubleType: double
FloatType: float
IntegerType: int
LongType: bigint
ShortType: smallint
StringType: string
TimestampType: timestamp

# selectExpr

#### SQL like expressions can be used for evaluation

In [32]:
orders.selectExpr('order_id||order_date||","||order_customer_id||","||order_status||"," as textdata ').show(1)

+--------------------+
|            textdata|
+--------------------+
|12013-07-25 00:00...|
+--------------------+
only showing top 1 row



# case
#### CASE as in SQL can be used

In [33]:
orders.selectExpr('CASE WHEN order_status in ("COMPLETE","CLOSED") THEN "COMPLETELD" WHEN order_status = "CANCELED" THEN "CANCEL" ELSE "NONE" END Derived_status').show()

+--------------+
|Derived_status|
+--------------+
|    COMPLETELD|
|          NONE|
|    COMPLETELD|
|    COMPLETELD|
|    COMPLETELD|
|    COMPLETELD|
|    COMPLETELD|
|          NONE|
|          NONE|
|          NONE|
|          NONE|
|    COMPLETELD|
|          NONE|
|          NONE|
|    COMPLETELD|
|          NONE|
|    COMPLETELD|
|    COMPLETELD|
|          NONE|
|          NONE|
+--------------+
only showing top 20 rows



# when
#### Dataframe equivalent of case 

In [4]:
orders.withColumn('Derived',when(orders.order_status.isin('COMPLETE','CLOSED'),'DONEEE').when(orders.order_status=='PENDING_PAYMENT','Gareeb').otherwise('Bhool Ja')).show()

+--------+--------------------+-----------------+---------------+--------+
|order_id|          order_date|order_customer_id|   order_status| Derived|
+--------+--------------------+-----------------+---------------+--------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|  DONEEE|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|  Gareeb|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|  DONEEE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|  DONEEE|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|  DONEEE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|  DONEEE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|  DONEEE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|Bhool Ja|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|  Gareeb|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|  Gareeb|
|      11|2013-07-25 00:0

# withColumnRenamed
#### columns can be renamed with this API, where the first argument is the existing column name and the second argument is the new name.

In [40]:
orders.withColumnRenamed('order_status','status_of_order').show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|status_of_order|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

# filter

#### filter as in SQL format

In [44]:
orders.filter?

Signature: orders.filter(condition)
Docstring:
Filters rows using the given condition.

:func:`where` is an alias for :func:`filter`.

:param condition: a :class:`Column` of :class:`types.BooleanType`
    or a string of SQL expression.

>>> df.filter(df.age > 3).collect()
[Row(age=5, name='Bob')]
>>> df.where(df.age == 2).collect()
[Row(age=2, name='Alice')]

>>> df.filter("age > 3").collect()
[Row(age=5, name='Bob')]
>>> df.where("age = 2").collect()
[Row(age=2, name='Alice')]

.. versionadded:: 1.3
File:      d:\documents\pyspark\spark-3.0.0\python\pyspark\sql\dataframe.py
Type:      method


In [41]:
orders.filter("order_status = 'COMPLETE'").show()
#Note: Single '=' as in SQL format

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      26|2013-07-25 00:00:...|             7562|    COMPLETE|
|      28|2013-07-25 00:00:...|              656|    COMPLETE|
|      32|2013-07-25 00:00:...|             3960|    COMPLETE|
|      35|2013-07-25 00:00:...|             4840|    COMPLETE|
|      45|2013-07-25 00:00:...|             2636|    COMPLETE|
|      56|2013-07-25 00:00:...|            10519|    CO

#### filter as in Dataframe format

In [48]:
orders.filter(orders.order_status == 'COMPLETE').show()
#Note: Equality operator '==' for dataframe filter expression 

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      26|2013-07-25 00:00:...|             7562|    COMPLETE|
|      28|2013-07-25 00:00:...|              656|    COMPLETE|
|      32|2013-07-25 00:00:...|             3960|    COMPLETE|
|      35|2013-07-25 00:00:...|             4840|    COMPLETE|
|      45|2013-07-25 00:00:...|             2636|    COMPLETE|
|      56|2013-07-25 00:00:...|            10519|    CO

#### multiple filters in SQL format filter

In [5]:
orders.filter("order_status = 'COMPLETE' OR order_status = 'CLOSED'").show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      12|2013-07-25 00:00:...|             1837|      CLOSED|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      18|2013-07-25 00:00:...|             1205|      CLOSED|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      24|2013-07-25 00:00:...|            11441|      CLOSED|
|      25|2013-07-25 00:00:...|             9503|      

#### multiple filters in DataFrame format

In [68]:
#orders.filter((orders.order_status == 'COMPLETE').__or__(orders.order_status == 'CLOSED')).show() 
orders.filter((orders.order_status == 'COMPLETE') | (orders.order_status == 'CLOSED')).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      12|2013-07-25 00:00:...|             1837|      CLOSED|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      18|2013-07-25 00:00:...|             1205|      CLOSED|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      24|2013-07-25 00:00:...|            11441|      CLOSED|
|      25|2013-07-25 00:00:...|             9503|      

###### Examples of few multiple filters

In [7]:
orders.filter("order_status in ('COMPLETE' ,'CLOSED') AND date_format(order_date,'yyyyMM') = '201308'").show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|    1297|2013-08-01 00:00:...|            11607|    COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|      CLOSED|
|    1299|2013-08-01 00:00:...|             7802|    COMPLETE|
|    1302|2013-08-01 00:00:...|             1695|    COMPLETE|
|    1304|2013-08-01 00:00:...|             2059|    COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|    COMPLETE|
|    1307|2013-08-01 00:00:...|             4474|    COMPLETE|
|    1309|2013-08-01 00:00:...|             2367|      CLOSED|
|    1312|2013-08-01 00:00:...|            12291|    COMPLETE|
|    1314|2013-08-01 00:00:...|            10993|    COMPLETE|
|    1315|2013-08-01 00:00:...|             5660|    COMPLETE|
|    1318|2013-08-01 00:00:...|             4212|    COMPLETE|
|    1319|2013-08-01 00:00:...|             3966|    CO

In [64]:
#orders.registerTempTable("df")
#spark.sql("select * from df where order_date like '2013-08%'").show()

orders.filter(orders.order_status.isin('COMPLETE','CLOSED') & orders.order_date.like('2013-08%')).show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|    1297|2013-08-01 00:00:...|            11607|    COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|      CLOSED|
|    1299|2013-08-01 00:00:...|             7802|    COMPLETE|
|    1302|2013-08-01 00:00:...|             1695|    COMPLETE|
|    1304|2013-08-01 00:00:...|             2059|    COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|    COMPLETE|
|    1307|2013-08-01 00:00:...|             4474|    COMPLETE|
|    1309|2013-08-01 00:00:...|             2367|      CLOSED|
|    1312|2013-08-01 00:00:...|            12291|    COMPLETE|
|    1314|2013-08-01 00:00:...|            10993|    COMPLETE|
|    1315|2013-08-01 00:00:...|             5660|    COMPLETE|
|    1318|2013-08-01 00:00:...|             4212|    COMPLETE|
|    1319|2013-08-01 00:00:...|             3966|    CO

In [13]:
orders.filter('order_customer_id >= 1000').filter('order_customer_id != 2321').filter(orders.order_id.between(1000,1999)).show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|    1001|2013-07-30 00:00:...|             6650|       COMPLETE|
|    1002|2013-07-30 00:00:...|             5170|SUSPECTED_FRAUD|
|    1003|2013-07-30 00:00:...|             7438|       COMPLETE|
|    1004|2013-07-30 00:00:...|            10701|       COMPLETE|
|    1005|2013-07-30 00:00:...|             7958|     PROCESSING|
|    1006|2013-07-30 00:00:...|             9028|       COMPLETE|
|    1007|2013-07-30 00:00:...|             1417|PENDING_PAYMENT|
|    1008|2013-07-30 00:00:...|             4331|         CLOSED|
|    1009|2013-07-30 00:00:...|             7886|       COMPLETE|
|    1010|2013-07-30 00:00:...|             2512|       COMPLETE|
|    1011|2013-07-30 00:00:...|             7214|       COMPLETE|
|    1012|2013-07-30 00:00:...|             3245|         CLOSED|
|    1013|

# join

#### simple inner join with one column mapping

In [15]:
orders.join?

Signature: orders.join(other, on=None, how=None)
Docstring:
Joins with another :class:`DataFrame`, using the given join expression.

:param other: Right side of the join
:param on: a string for the join column name, a list of column names,
    a join expression (Column), or a list of Columns.
    If `on` is a string or a list of strings indicating the name of the join column(s),
    the column(s) must exist on both sides, and this performs an equi-join.
:param how: str, default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
    ``full``, ``fullouter``, ``full_outer``, ``left``, ``leftouter``, ``left_outer``,
    ``right``, ``rightouter``, ``right_outer``, ``semi``, ``leftsemi``, ``left_semi``,
    ``anti``, ``leftanti`` and ``left_anti``.

The following performs a full outer join between ``df1`` and ``df2``.
>>> from pyspark.sql.functions import desc
>>> df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height)                 .sort(desc("name")).collect()
[Ro

In [25]:
orders.join(order_items, orders.order_id==order_items.order_item_order_id).show()

+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_customer_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|            1|                  1|                  957|                  1|             299.98|                  299.98|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|

To perform join but get results from only the left dataframe.

In [29]:
orders.join(orders, ['order_id'],how='left_semi').show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

#### inner join with multiple columns

In [18]:
orders.join(order_items, [orders.order_id==order_items.order_item_order_id, orders.order_id==order_items.order_item_order_id]).show()

+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_customer_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|            1|                  1|                  957|                  1|             299.98|                  299.98|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|

In [ ]:
#### left join

In [19]:
customers.join(orders, customers.customer_id==orders.order_customer_id, 'left').show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------------+---------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|order_id|          order_date|order_customer_id|   order_status|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------------+---------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|   22945|2013-12-13 00:00:...|                1|       COMPLETE|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|   67863|2013-11-30 00:00

#### right join

In [20]:
customers.join(orders, customers.customer_id==orders.order_customer_id, 'right').show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------------+---------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|order_id|          order_date|order_customer_id|   order_status|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------------+---------------+
|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicago|            IL|           60625|       2|2013-07-25 00:00

#### leftanti join - Join to fetch records which exists only in one table
In the below case it will fetch the records which are present only in the orders and not in order_items

In [22]:
customers.join(orders, orders.order_customer_id==customers.customer_id, 'leftanti').show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|      customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------------+--------------+----------------+
|        219|          Mary|       Harrell|     XXXXXXXXX|        XXXXXXXXX|9016 Foggy Robin ...|             Denver|            CO|           80219|
|        339|          Mary|        Greene|     XXXXXXXXX|        XXXXXXXXX|     4271 Hazy Close|         Long Beach|            CA|           90805|
|        469|         Randy|         Smith|     XXXXXXXXX|        XXXXXXXXX|252 Golden Goose ...|South San Francisco|            CA|           94080|
|       1187|       Dorothy|       Vazquez|     XXXXXXXXX|        XXXXXXXXX| 363 Green Goose Run|   

#### crossJoin

In [ ]:
orders.crossJoin(customers.filter(customers.customer_id == 1)).show()

# distinct
#### distinct will come at the end after the select

In [ ]:
orders.select('order_status').distinct().show()

# countDinstinct
#### countDistinct will be handy 

In [ ]:
orders.select(countDistinct('order_status')).show()

# orderBy/sort
#### Note: sort is just an alias to orderBy

In [ ]:
orders.orderBy(orders.order_date, orders.order_status.desc()).show()

Note: Only absolute column name notation or col() works while mentioning in descending order desc()

In [ ]:
orders.orderBy('order_date',col('order_status').desc()).show()

# drop

drop() will only take just the column names and it will not take any other expressions for the column

In [ ]:
order_items.join(products,order_items.order_item_product_id==products.product_id).drop('product_price','product_description','product_image').show()

In [ ]:
#order_items.join(products, order_item_product_id==products.product_id).drop(products.product_price, products.product_description, products.product_image).show()
# Specifying full name of the column will not work 